<a href="https://colab.research.google.com/github/ARJUN108-verma/Vector-Database/blob/main/7_Similarity_Search_and_querying_the_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Similarity search and querying the data:-

In [ ]:
import pandas as pd
import os
from pinecone import Pinecone, ServerlessSpec
from dotenv import load_dotenv, find_dotenv
import pinecone
from sentence_transformers import SentenceTransformer

In [ ]:
files = pd.read_csv("course_descriptions.csv", encoding = "ANSI")

In [ ]:
def create_course_description(row):
    return f'''The course name is {row["course_name"]}, the slug is {row["course_slug"]},
            the technology is {row["course_technology"]} and the course topic is {row["course_topic"]}'''

In [ ]:
pd.set_option('display.max_rows', 106)
files['course_description_new'] = files.apply(create_course_description, axis = 1)
print(files["course_description_new"])

In [ ]:
%load_ext dotenv
%dotenv

In [ ]:
load_dotenv(find_dotenv(), override = True)

In [ ]:
pc = Pinecone(api_key = os.environ.get("PINECONE_API_KEY"), environment = os.environ.get("PINECONE_ENV"))

In [ ]:
index_name = "my-index"
dimension = 384
metric = "cosine"

In [ ]:
if index_name in [index.name for index in pc.list_indexes()]:
    pc.delete_index(index_name)
    print(f"{index_name} succesfully deleted.")
else:
     print(f"{index_name} not in index list.")

In [ ]:
pc.create_index(
    name = index_name,
    dimension = dimension,
    metric = metric,
    spec = ServerlessSpec(
        cloud = "aws",
        region = "us-east-1")
    )

In [ ]:
index = pc.Index(index_name)

Embedding the data

In [ ]:
model = SentenceTransformer('multi-qa-distilbert-cos-v1')

In [ ]:
def create_embeddings(row):
    combined_text = ' '.join([str(row[field]) for field in ['course_description', 'course_description_new', 'course_description_short']])
    embedding = model.encode(combined_text, show_progress_bar = False)
    return embedding

In [ ]:
files["embedding"] = files.apply(create_embeddings, axis = 1)

In [ ]:
vectors_to_upsert = [(str(row["course_name"]), row["embedding"].tolist()) for _, row in files.iterrows()]
index.upsert(vectors = vectors_to_upsert)

In [ ]:
print("Data upserted to Pinecone index")